In [67]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
import csv
import pandas as pd
hades = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

In [104]:
def scrape_detik(hal):
    data = []
    global hades
    a = 1
    for page in range(1,hal):
        url = f'https://www.detik.com/search/searchall?query=pemilu+capres&sortby=time&page={page}'
        ge = req.get(url,hades).text
        sop = bs(ge,'lxml')
        li = sop.find('div',class_='list media_rows list-berita')
        lin = li.find_all('article')
        for x in lin:
            link = x.find('a')['href']
            date = x.find('a').find('span',class_='date').text.replace('WIB','').replace('detikNews','').split(',')[1]
            headline = x.find('a').find('h2').text
            ge_ = req.get(link,hades).text
            sop_ = bs(ge_,'lxml')
            content = sop_.find_all('div',class_='detail__body-text itp_bodycontent')
            for x in content:
                x = x.find_all('p')
                y  = [y.text for y in x ]
                content_ = ''.join(y).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                
                a += 1
                data.append({'Date': date, 'Headline': headline, 'Link': link, 'Content': content_})

    df = pd.DataFrame(data)
    return df

In [108]:
df = scrape_detik(1000)
writer = pd.ExcelWriter('politik.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.close()